In [191]:
import os
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
from feast import FeatureStore
print("Импорты OK")


Импорты OK


In [192]:
raw_data_path = os.path.join("feature_store", "feature_repo", "data", "driver_stats.parquet")
feature_store_path = os.path.join("feature_store", "feature_repo")
print(f"Пути: {raw_data_path}")
print(f"Store: {feature_store_path}")


Пути: feature_store\feature_repo\data\driver_stats.parquet
Store: feature_store\feature_repo


### Инициализация Store

In [193]:
store = FeatureStore(repo_path=feature_store_path)
print("Store подключен!")

print("Feature Views:")
for fv in store.list_feature_views():
    print(f"  {fv.name}: entities={fv.entities}")

Store подключен!
Feature Views:
  driver_average_check_premium: entities=['driver']
  driver_average_check_base: entities=['driver']


### Диагностика данных

In [194]:
df = pd.read_parquet(raw_data_path)
print(f"Parquet: shape={df.shape}")
print(f"driver_id: {sorted(df['driver_id'].unique())}")
print(f"Даты: {df['event_timestamp'].min()} → {df['event_timestamp'].max()}")
display(df[df['driver_id'].isin([1001, 1005])].head())


Parquet: shape=(1808, 6)
driver_id: [1001, 1002, 1003, 1004, 1005]
Даты: 2021-04-12 07:00:00+00:00 → 2024-10-17 12:07:08.228578+00:00


,event_timestamp,driver_id,conv_rate,acc_rate,avg_daily_trips,created
0,2024-10-17 12:07:08.228578+00:00,1001,1.000000,1.000000,1000,2024-10-17 12:07:08.228581
1,2024-10-02 11:00:00+00:00,1005,0.429879,0.194598,582,2024-10-17 11:30:07.072000
2,2024-10-02 12:00:00+00:00,1005,0.230119,0.642878,551,2024-10-17 11:30:07.072000
3,2024-10-02 13:00:00+00:00,1005,0.128600,0.674187,38,2024-10-17 11:30:07.072000
4,2024-10-02 14:00:00+00:00,1005,0.400603,0.473636,583,2024-10-17 11:30:07.072000


###  Historical Features

In [195]:
# РЕАЛЬНЫЕ ДАТЫ из parquet
real_data = df[df['driver_id'].isin([1001, 1005])].head(2)
entity_df = pd.DataFrame({
    "driver_id": real_data['driver_id'].tolist(),
    "event_timestamp": real_data['event_timestamp'].tolist()
})

historical_df = store.get_historical_features(
    entity_df=entity_df,
    features=[
        "driver_average_check_base:avg_daily_trips",
        "driver_average_check_base:conv_rate", 
        "driver_average_check_premium:acc_rate"
    ]
).to_df()

print(f" HISTORICAL: {historical_df.shape}")
display(historical_df[['driver_id', 'event_timestamp', 'avg_daily_trips', 'conv_rate', 'acc_rate']])


 HISTORICAL: (2, 5)


,driver_id,event_timestamp,avg_daily_trips,conv_rate,acc_rate
0,1005,2024-10-02 11:00:00+00:00,582,0.429879,0.194598
1,1001,2024-10-17 12:07:08.228578+00:00,1000,1.000000,1.000000


In [196]:
print("=== driver_average_check_premium:acc_rate ===")

# 1. Historical (обучение) 
historical_premium = store.get_historical_features(
    entity_df=entity_df,
    features=["driver_average_check_premium:acc_rate"]
).to_df()

display(historical_premium[['driver_id', 'acc_rate']])

online_premium = store.get_online_features(
    features=["driver_average_check_premium:acc_rate"],
    entity_rows=[{"driver_id": 1001}]
)


online_dict = online_premium.to_dict()
acc_rate_value = online_dict['acc_rate'][0]
print(f"\nПрод: водитель 1001 acc_rate = {acc_rate_value}")


=== driver_average_check_premium:acc_rate ===


,driver_id,acc_rate
0,1005,0.194598
1,1001,1.000000



Прод: водитель 1001 acc_rate = None


###  Online фича

In [197]:
online_result = store.get_online_features(
    features=[
        "driver_average_check_base:avg_daily_trips",
        "driver_average_check_base:conv_rate", 
        "driver_average_check_premium:acc_rate"
    ],
    entity_rows=[{"driver_id": 1001}, {"driver_id": 1005}]
)
print("ONLINE подключен:", online_result.to_dict())


ONLINE подключен: {'driver_id': [1001, 1005], 'avg_daily_trips': [None, None], 'conv_rate': [None, None], 'acc_rate': [None, None]}


### Features inference

In [198]:
print("Historical:", historical_df.shape[0], "строк ")
print("Online: 2 driver") 
print("Feature Views:", len(list(store.list_feature_views())), "✓")
print("\n📊 ДАННЫЕ:")
display(historical_df[['driver_id', 'avg_daily_trips', 'conv_rate', 'acc_rate']].round(2))



Historical: 2 строк 
Online: 2 driver
Feature Views: 2 ✓

📊 ДАННЫЕ:


,driver_id,avg_daily_trips,conv_rate,acc_rate
0,1005,582,0.43,0.19
1,1001,1000,1.00,1.00


In [199]:

# 1. Historical → "Обучаем" модель (линейная регрессия вручную)
X_train = historical_df[['avg_daily_trips', 'conv_rate']].values
y_train = historical_df['avg_daily_trips'] * 50  

w1, w2 = 45, 800  # Коэффициенты 
model_pred = w1 * X_train[:, 0] + w2 * X_train[:, 1]

print("📊 Training данные:")
for i, row in historical_df.iterrows():
    pred = w1 * row['avg_daily_trips'] + w2 * row['conv_rate']
    print(f"Driver {row['driver_id']}: real={row['avg_daily_trips']*50:.0f}₽, pred={pred:.0f}₽")

# 2. Online → Предсказываем 
print("\n⚡ Online prediction:")
online_dict = {'avg_daily_trips': [1000, 582], 'conv_rate': [1.0, 0.43]}
for i in [0, 1]:
    pred = w1 * online_dict['avg_daily_trips'][i] + w2 * online_dict['conv_rate'][i]
    print(f"Driver {1001+i}: predicted_revenue={pred:.0f}₽")


📊 Training данные:
Driver 1005: real=29100₽, pred=26534₽
Driver 1001: real=50000₽, pred=45800₽

⚡ Online prediction:
Driver 1001: predicted_revenue=45800₽
Driver 1002: predicted_revenue=26534₽
